# Homework assignment 3

### Student 1
- Name: [[Gerard Planella Fontanillas]]
- Student ID: [[14244950]]

### Student 2
- Name: [[Veljko Kovac]]
- Student ID: [[11842237]]

## 1. Instructions

Fill in your name(s) and student ID(s) in the cell above. If you work alone, you may remove the part about "Student 2".

Then finish this notebook by writing an ASP encoding of a planning problem. Details about the planning problem and more detailed instructions for what to do can be found below.

## 2. The problem

In this homework assignment, you will write an ASP program to solve the
following (toy) planning problem.
This problem is about finding a way to deliver some packages from warehouses
to specified delivery locations, using several (electric) trucks.

The problem works in time steps. At each time step, each truck can perform exactly
one action—move to an adjacent node, wait, (un)load a package, and so on (more details below).
The problem operates on a map that consists of a graph with nodes and edges.
All nodes and edges are single capacity, meaning that only a single truck can be on a node or travel on an edge at each time step.
The task is to find a plan (a sequence of actions, one per truck per time step)
after which the goal state is achieved: all packages are delivered, the trucks are empty (i.e., without packages) and are located on a charging spot.

### The initial state

The initial state of the world consists of:
- a road network in the form of an undirected graph;
- the location (on the network) and initial load (number of available packages) of one or more warehouses;
- the location (on the network) and initial demand (number of packages to be delivered) of one or more delivery locations;
- the location (on the network) of one or more charging points; and
- the initial location, the initial load (how many packages aboard), the maximum load capacity, the initial battery level, and the maximum battery level of one or more trucks.

(These are all given as part of the input.)

### The actions (and their conditions and effects)

At each time step, each truck performs exactly one of the following actions:
- It can **wait**—the truck stays in the current location, the battery level stays the same.
- It can **move** to an adjacent node—the battery level goes down by 1 unit.
- If it is at a warehouse, the warehouse is not empty, and the truck still has capacity, it can **load** one single package from the warehouse into the truck—the battery level stays the same.
- If it is at a delivery location that still needs some packages, and the truck has at least one package, it can **unload** one single package at the delivery location—the battery level stays the same.
- If it is at a charging station, and the battery is not at full capacity, it can **charge**—the battery level increases by 1 unit.

The following additional restrictions have to be taken into account:
- If the battery level of a truck is 0, it cannot move to another node.
- At each location, there can be at most one truck at the same time.
- Trucks cannot pass each other on a piece of road—e.g., trucks at adjacent locations cannot swap places in a single time step.

### The goal

The goal is to achieve a state that has all of the following properties:
- All required deliveries must have been performed.
- Each truck has no packages aboard.
- Each truck is located at a charging station.

### The task

Finish this notebook by encoding this problem into ASP.
Make sure that your encoding works for any given input (that is of the right format).
That is, for each input, your code constructs an answer set program
whose answer sets correspond to the (shortest) sequences
of actions that achieve the goal from the initial state of the world—if the goal state can be reached within a given maximum number of steps.

## 3. Setting up
We begin with setting up and importing some relevant things.

In [ ]:
### DO NOT CHANGE THIS CELL

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

## 4. Input
As input for our planning problem, we get a road network in the form of an undirected graph, specified as:
- a list of nodes
- a list of pairs of nodes, specifying the edges of the graph (these are to be interpreted as undirected)

Additionally, we get:
- a list of dictionaries specifying the factories (with their location on the graph and initial load)
- a list of dictionaries specifying the delivery points (with their location and initial demand)
- a list of dictionaries specifying the charging points (with their location)
- a list of dictionaries specifying the trucks, with:
  * their initial location
  * their initial load
  * their (load) capacity
  * their initial charge level
  * their maximum charge capacity

(See the example input below for the exact format of these dictionaries.)

Moreover, as part of the input, we get a maximum number of time steps, within which the goal should be reached.

### Example input

In [ ]:
### YOU MAY CHANGE THIS CELL TO TEST YOUR SOLUTION ON DIFFERENT INPUTS

nodes = list(range(1, 6+1))
edges = [
    (1,2),
    (2,3),
    (3,4),
    (4,5),
    (2,6),
    (3,6),
]
factories = [
    {
        'location': 1,
        'load': 10,
    },
]
delivery_points = [
    {
        'location': 4,
        'demand': 2,
    },
    {
        'location': 5,
        'demand': 2,
    },
]
charging_points = [
    {
        'location': 2,
    },
    {
        'location': 6,
    },
]
trucks = [
    {
        'location': 1,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
    {
        'location': 2,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
]

num_time_steps = 40

### Checking input assumptions

We will only consider inputs that have the following properties:
- no node is both a warehouse and a delivery location;
- no node is both a warehouse and a charging location; and
- no node is both a delivery and a charging location.

This means that your solution only needs to work for inputs that satisfy these assumptions.

The following code will check these assumptions, and throw an exception if they are not satisfied.

In [ ]:
### DO NOT CHANGE THIS CELL

warehouse_locations = [
    warehouse_info['location']
    for warehouse_info in factories
]
truck_locations = [
    truck_info['location']
    for truck_info in trucks
]
delivery_locations = [
    dp_info['location']
    for dp_info in delivery_points
]
charging_locations = [
    cp_info['location']
    for cp_info in charging_points
]

class InvalidInputError(Exception):
    pass

intersection = [location for location in warehouse_locations if location in delivery_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a delivery location')

intersection = [location for location in warehouse_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a charging location')

intersection = [location for location in delivery_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a delivery and a charging location')

## 5. Assignment: writing an ASP program for finding solutions

Write down your encoding of the problem into ASP in cells in this section.
Make sure to interleave your code cells with appropriate text/markdown cells where you explain what you are doing.

In order to make the solution pretty-printing machinery in this notebook work, you should:
- use a predicate `do/3` to capture what actions the trucks take
  (where `do(T,Tr,A)` indicates that at time `T`, truck `Tr` takes action `A`); and
- use a predicate `truck_at/2` to capture the initial state of the trucks
  (where `truck_at(Tr,N)` indicates that the initial state of truck `Tr` is node `N`).

In [93]:
### FILL IN YOUR SOLUTION HERE
### (AND ADD MORE CELLS DIRECTLY BELOW THIS ONE)

# Your program should go in the variable `asp_program`.

# The dummy below is intended to remind you to use do/3 and truck_at/2,
# to make the printing machinery work nicely.

asp_program = f"#const t={num_time_steps}.\n"

asp_program+="""
possible_time(1..t).
{ time(T): possible_time(T) }.
:- not time(1).
:-possible_time(T), possible_time(T+1), not time(T), time(T+1).

max_time(MaxTime) :- MaxTime = #max {T:time(T)}.
"""

for node in nodes:
    asp_program+=f"node({node}).\n"

for edge in edges:
    asp_program+=f"edge({edge[0]}, {edge[1]}).\n"

for factory in factories:
    asp_program+=f"""
factory({factory['location']}).
packages_at_factory({factory['location']}, {factory['load']}).
"""

for delivery_point in delivery_points:
    asp_program+=f"""
delivery_point({delivery_point['location']}).
demand_at_delivery_point({delivery_point['location']}, {delivery_point['demand']}).
"""

for charging_point in charging_points:
    asp_program+=f"charging_point({charging_point['location']}).\n"

for id, truck in enumerate(trucks):
    asp_program+=f"""
truck({id}).
truck_at({id}, {truck['location']}).
truck_capacity({id}, {truck['capacity']}).
truck_load({id}, {truck['load']}).
truck_charge_capacity({id},{truck['charge capacity']}).
truck_charge_level({id}, {truck['charge level']}).
"""

asp_program += """

% Edges are undirected 
edge(X,Y) :- edge(Y,X).

% ---------------------- Actions -----------------------------------

action(move(PosStart, PosEnd)) :- edge(PosStart, PosEnd).
action(load(Pos))   :- factory(Pos).
action(charge(Pos)) :- charging_point(Pos).
action(unload(Pos)) :- delivery_point(Pos).
action(wait).

% Generate all possible Actions for each truck at each time step
1 { do(T,Tr, A) : action(A) } 1 :- time(T), truck(Tr). 

% ---------------------- Init States-----------------------------------
state(1, truck_at(Tr,Pos)) :- truck_at(Tr, Pos).
state(1, truck_charge_level(TR,L)) :- truck_charge_level(TR,L).
state(1, truck_load(Tr,Load)) :- truck_load(Tr,Load).
state(1, packages_at_factory(Pos,Load)) :- packages_at_factory(Pos,Load).
state(1, demand_at_delivery_point(Pos,Demand)) :- demand_at_delivery_point(Pos,Demand).

% ---------------------- Truck Movement ---------------------------------
state(T+1,truck_at(Tr, Pos2)) :- do(T,Tr,move(Pos1, Pos2)), state(T,truck_at(Tr, Pos1)), truck(Tr). 
state(T+1,truck_at(Tr, Pos)) :- do(T,Tr,_), not do(T,Tr,move(_,_)), state(T,truck_at(Tr,Pos)), truck(Tr).  % Dont move if you dont move
state(T+1,truck_charge_level(Tr,ChargeLevel)) :- do(T,Tr,_), not do(T,Tr, charge(_)), not do(T,Tr, move(_,_)), state(T,truck_charge_level(Tr,ChargeLevel)), truck(Tr).
state(T+1,truck_charge_level(Tr, ChargeLevel - 1)) :-  do(T,Tr,move(_, _)), state(T,truck_charge_level(Tr, ChargeLevel)), truck(Tr).
state(T+1,truck_charge_level(Tr,ChargeLevel+1)) :- do(T,Tr,charge(_)), state(T, truck_charge_level(Tr,ChargeLevel)), truck(Tr).


:- state(T,truck_at(Tr1,Pos)), state(T,truck_at(Tr2,Pos)), Tr1 != Tr2, node(Pos), truck(Tr1), truck(Tr2). % No truck on same node
:- do(T,ID,charge(_)), state(T,truck_charge_level(Tr,ChargeLevel1)), truck_charge_capacity(Tr,ChargeLevel2), truck(Tr), ChargeLevel1 == ChargeLevel2.
:- do(T,Tr1,move(Pos1, Pos2)), do(T,Tr2,move(Pos2, Pos1)), state(T, truck_at(Tr1, Pos1)), state(T, truck_at(Tr2, Pos2)), truck(Tr1), truck(Tr2). %Trucks cant move past each other
:- do(T,Tr,move(_, _)), state(T,truck_charge_level(Tr, ChargeLevel)), truck(Tr), ChargeLevel == 0. % Can't move with no battery
:- do(T,Tr,move(Pos1,_)), state(T,truck_at(Tr,Pos2)), truck(Tr), Pos2 != Pos1. % Can't perform an action from somewhere if truck isn't there
:- do(T,Tr,charge(Pos1)), state(T,truck_at(Tr,Pos2)), truck(Tr), Pos2 != Pos1. % Can't perform an action from somewhere if truck isn't there

% ----------------------- Package Unloading ------------------------

state(T+1, truck_load(Tr,Load-1)) :- state(T,truck_load(Tr,Load)), do(T,Tr,unload(_)).
state(T+1, truck_load(Tr,Load))   :- state(T,truck_load(Tr,Load)), do(T,Tr,move(_,_)).
state(T+1, truck_load(Tr,Load))   :- state(T,truck_load(Tr,Load)), do(T,Tr,wait).
state(T+1, truck_load(Tr,Load))   :- state(T,truck_load(Tr,Load)), do(T,Tr,charge(_)).
state(T+1, demand_at_delivery_point(Pos,Demand)) :- state(T,demand_at_delivery_point(Pos,Demand)), do(T,_,_) , not do(T,_,unload(Pos)), delivery_point(Pos).
state(T+1, demand_at_delivery_point(Pos,Demand-1)) :- state(T,demand_at_delivery_point(Pos,Demand)), do(T,_,unload(Pos)), delivery_point(Pos).
state(T+1, packages_at_factory(Pos,Load)) :-  state(T,packages_at_factory(Pos,Load)), do(T,_,_), not do(T,_,load(Pos)), factory(Pos).


:- do(T,Tr,unload(_)), state(T, truck_load(Tr,Load)), Load == 0. % Empty truck can't unload.
:- do(T,Tr,unload(_)), state(T,demand_at_delivery_point(Pos,Demand)), delivery_point(Pos), Demand == 0. % Can't unload at delivery point with no demand.
:- do(T,Tr,unload(Pos1)), state(T,truck_at(Tr,Pos2)), Pos2 != Pos1. % Can't perform an action from somewhere if truck isn't there

% ----------------------- Package Loading ------------------------
% If a package is loaded, factory load decreases
state(T+1,packages_at_factory(Pos,Load-1)) :- do(T,_ ,load(Pos)), state(T,packages_at_factory(Pos,Load)), factory(Pos).
% Truck load increases when loading
state(T+1,truck_load(Tr,Load+1)) :- do(T,Tr,load(_)),state(T,truck_load(Tr,Load)).

:- do(T,Tr,load(Pos)), state(T,packages_at_factory(Pos, FactoryLoad)), factory(Pos), FactoryLoad == 0. % Can't Load in a factory if factory has no packages.
:- do(T,Tr,load(_)), state(T,truck_load(Tr,Load)),truck_capacity(Tr, Capacity), Load == Capacity. % If truck is full, it cant load a package
:- do(T,Tr,load(Pos1))  , state(T,truck_at(Tr,Pos2)), Pos2 != Pos1. % Can't perform an action from somewhere if truck isn't there

% ----------------------- Goals ------------------------

all_trucks_charging(T) :- possible_time(T), state(T, truck_at(_, Pos)) : charging_point(Pos).
all_delivery_points_full(T) :- possible_time(T), state(T, demand_at_delivery_point(Pos, 0)) : delivery_point(Pos).
all_trucks_empty(T) :- possible_time(T), state(T, truck_load(Tr,0)) : truck(Tr).

%Setting goal to T-1 indicates that current actions will lead to a next state that has already reached the goal.
goal_reached(T-1) :-  state(T, _), all_trucks_charging(T), all_delivery_points_full(T), all_trucks_empty(T).

%We require that the goal is reached at the end
:- max_time(M), not goal_reached(M).
%Once the goal is reached, we stop the clock
:- max_time(M), goal_reached(T), time(T), T < M.

% ----------------------- Optimisation ------------------------

#minimize { 1,T : state(T, _) }. %Shortest sequence of states

% ----------------------- Visualisation ------------------------

#show do/3.
#show truck_at/2.
"""

print(asp_program)

#const t=40.

possible_time(1..t).
{ time(T): possible_time(T) }.
:- not time(1).
:-possible_time(T), possible_time(T+1), not time(T), time(T+1).

max_time(MaxTime) :- MaxTime = #max {T:time(T)}.
node(1).
node(2).
node(3).
node(4).
node(5).
node(6).
edge(1, 2).
edge(2, 3).
edge(3, 4).
edge(4, 5).
edge(2, 6).
edge(3, 6).

factory(1).
packages_at_factory(1, 10).

delivery_point(4).
demand_at_delivery_point(4, 2).

delivery_point(5).
demand_at_delivery_point(5, 2).
charging_point(2).
charging_point(6).

truck(0).
truck_at(0, 1).
truck_capacity(0, 1).
truck_load(0, 0).
truck_charge_capacity(0,10).
truck_charge_level(0, 2).

truck(1).
truck_at(1, 2).
truck_capacity(1, 1).
truck_load(1, 0).
truck_charge_capacity(1,10).
truck_charge_level(1, 2).


% Edges are undirected 
edge(X,Y) :- edge(Y,X).

% ---------------------- Actions -----------------------------------

action(move(PosStart, PosEnd)) :- edge(PosStart, PosEnd).
action(load(Pos))   :- factory(Pos).
action(charge(Pos)) :- charging_poi

In [ ]:
def print_answer_sets(program, num_answer_sets=0):
    
    # Load the answer set program, and call the grounder
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}\n".format(", ".join(sorted_model)))
    
    # Ask clingo to find some number of models
    # (using an upper bound of 0 gives all models)
    control.configuration.solve.models = num_answer_sets
    
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

print_answer_sets(asp_program)

## 6. Finding and outputting solutions

The following cell contains some code that will be used to find (optimal) answer sets for your ASP program, to translate this to a solution for the planning problem, and to display this solution in a human-readable format.

You don't need to finish or change this cell in your solution.

In [ ]:
### DO NOT CHANGE THIS CELL

# Some machinery to (dynamically) print things in the notebook
log_str = ""
def reset_log():
    global log_str
    log_str = ""
def log(msg):
    global log_str
    print(msg, flush=True)
    log_str += f"{msg}\n"
def print_log():
    global log_str
    print(log_str, end='', flush=True)
    
def display_solution(model):
    """
    Displays a solution specified by the answer set
    in a human-readable format.
    
    Assumes that the answer set program uses do/3
    to specify actions, and uses truck_at/2 to
    specify the initial locations of the trucks.
    """
    actions = {}
    truck_locations = {}
    for atom in model.symbols(atoms=True):
        if atom.name == "do":
            time = atom.arguments[0].number
            truck = str(atom.arguments[1])
            action = str(atom.arguments[2])
            if time not in actions:
                actions[time] = {}
            actions[time][truck] = action
        elif atom.name == "truck_at":
            truck_locations[str(atom.arguments[0])] = str(atom.arguments[1])
    solution_str = "\n"
    solution_str += " "*5 + "".join([
        f"{truck:<5} @ {truck_locations[truck]:<10}"
        for truck in sorted(truck_locations)
    ]) + "\n"
    solution_str += "-"*(5+18*len(truck_locations)) + "\n"
    for time in sorted(actions):
        time_str = f"{time}:"
        solution_str += f"{time_str:<4} " + "".join([
            f"{actions[time][truck]:<18}"
            for truck in sorted(truck_locations)
        ]) + "\n"
    return solution_str

def find_and_output_solution(
    asp_program,
    timeout=10,
    num_threads=1,
):
    """
    Calls the ASP solver to find answer sets,
    and translates these to solutions,
    which are printed in human-readable format.
    """

    reset_log()
    log("..Grounding..")
    
    control = clingo.Control([f"--parallel-mode={num_threads}"])
    control.add("base", [], asp_program)
    control.ground([("base", [])])

    control.configuration.solve.models = 1
    control.configuration.solve.opt_mode = "optN"
    
    def interpret_model(model):
        clear_output(wait=True)
        print_log()
        print(display_solution(model))

    if timeout > 0:
        log(f"..Solving with timeout {timeout}s..")
        with control.solve(
            async_=True,
            on_model=lambda model: interpret_model(model),
        ) as handle:
            finished = handle.wait(timeout)
            if not finished:
                log("..Stopped after timeout..")
            else:
                log("..Stopped..")
                time = control.statistics['summary']['times']['total']
                log(f"..Total solving time: {time:.2f} seconds..")
    else:
        log("..Solving..")
        control.solve(
            on_model=lambda model: interpret_model(model),
        )
        log("..Stopped..")
        time = control.statistics['summary']['times']['total']
        log(f"..Total solving time: {time:.2f} seconds..")

## 7. For your convenience: visualizing (part of) the input

For your (and our) convenience, let's visualize the input to the problem. The road network will be displayed as a graph. Nodes with a warehouse are blue. Nodes with a delivery location are green. Nodes with a charging point are purple. All other nodes are red. Nodes where there is a truck initially are labelled with "T".

Note that this doesn't display all the information about the input. For example, the truck (maximum) battery level and (maximum) load is not displayed, and neither is the load/demand of the warehouses and delivery locations.

In [ ]:
### NO NEED TO CHANGE THIS CELL
### (UNLESS THE IMAGE DOESN'T SHOW WELL FOR SOME INPUT YOU ARE USING)

plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = (8, 6)
graph = nx.Graph()

ax = plt.gca()
ax.margins(0.08)

options = {
    "edgecolors": "tab:gray",
    "node_size": 2000,
    "font_color": "white",
    "font_size": 14,
}
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)
layout = nx.spring_layout(graph, seed=42)

labels = {
    node:
    (f"{node} T" if node in truck_locations else f"{node}")
    for node in nodes
}

def node_color(node):
    if node in warehouse_locations:
        return "tab:blue"
    elif node in delivery_locations:
        return "tab:green"
    elif node in charging_locations:
        return "tab:purple"
    else:
        return "tab:red"
colors = [
    node_color(node)
    for node in nodes
]

nx.draw(
    graph,
    layout,
    ax=ax,
    node_color=colors,
    labels=labels,
    with_labels=True,
    **options
)

plt.show()

## 8. Finding and representing solutions

Finally, let's call the solver and display the solution that it might find.

Note that it will display a solution when it finds one, even if it then keeps searching for a better solution. Subsequent better (shorter) solutions will then be displayed, always displaying only the current best solution. Finally, after the solving stopped (possibly due to a time out), it will display "..Stopped.." below the best found solution.

In [92]:
### NO NEED TO CHANGE THIS CELL
### (UNLESS YOU WANT TO SET A SOLVING TIMEOUT
###  OR USE A DIFFERENT NUMBER OF PARALLEL THREADS)

find_and_output_solution(
    asp_program,
    timeout=0,
    num_threads=4,
)

..Grounding..
..Solving..

     0     @ 1         1     @ 2         
-----------------------------------------
1:   load(1)           charge(2)         
2:   move(1,2)         move(2,6)         
3:   charge(2)         charge(6)         
4:   charge(2)         charge(6)         
5:   charge(2)         charge(6)         
6:   charge(2)         wait              
7:   charge(2)         charge(6)         
8:   move(2,3)         move(6,2)         
9:   move(3,4)         move(2,1)         
10:  move(4,5)         load(1)           
11:  unload(5)         move(1,2)         
12:  move(5,4)         charge(2)         
13:  move(4,3)         move(2,6)         
14:  move(3,2)         charge(6)         
15:  charge(2)         charge(6)         
16:  charge(2)         charge(6)         
17:  move(2,1)         move(6,3)         
18:  load(1)           move(3,4)         
19:  move(1,2)         unload(4)         
20:  charge(2)         move(4,3)         
21:  move(2,6)         move(3,2)         
22:  ch